In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
mh_data=pd.read_csv('stemmed_data.csv',encoding='latin1')

In [3]:
X=mh_data.drop(columns='class',axis=1)
Y=mh_data['class']

In [4]:
#stemming
port_stem=PorterStemmer()
def stemming(text):
    text=text.lower()
    text=re.sub('\[.*?\]', '',text)
    text=re.sub("\\W"," ",text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]'% re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    text=text.split()
    text=[port_stem.stem(word) for word in text if not word in stopwords.words('english')]
    text=' '.join(text)
    return text

In [5]:
X=mh_data['text'].values
Y=mh_data['class'].values

In [6]:
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [7]:
from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
mh_data['class']=label_encoder.fit_transform(mh_data['class'])

In [8]:
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.2)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=10)
model.fit(X_train, Y_train)

RandomForestClassifier(max_depth=10)

In [11]:
#from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(n_neighbors=9)
#model=model.fit(X_train,Y_train)

In [12]:
#from sklearn.tree import DecisionTreeClassifier 
#model=DecisionTreeClassifier(criterion="entropy",max_depth=2000)
#model=model.fit(X_train,Y_train)

In [13]:
#model=LogisticRegression(solver='newton-cg', max_iter=100000)
#model.fit(X_train,Y_train)

In [14]:
from joblib import Parallel, delayed
import joblib

In [15]:
joblib.dump(model, 'saved_model.pkl')

['saved_model.pkl']

In [16]:
X_new=pd.Series(input())
print(type(X_new))
X_new=X_new.apply(stemming)
print(type(X_new))

I hurt another person because I am too ungrateful that I am alive.I feel so messed up.  I was venting to someone on chat, and I told him what I am planning to do. Killing myself sometime this week.  He told me he has lung cancer, only 5% chances to live, and he wants to live. He called me ungrateful fucking shit because I want to kill myself only because I don't feel happy with anything anymore, how he wants my life. I hurt him so badly. I apologized like 100 times. I know I can't just apologized about that. He sent me a video with himself calling me all these things, and he was literally crying his eyes out.  I hate myself. I did hurt him so badly..  I still want to die, even more now. I cut myself for the first time ever today. Just to feel a little better.  I can't forgive myself now.  I am still planning on killing myself, I guess. He told me to never talk to him again. 
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [17]:
X_new=X_new.values
vect=TfidfVectorizer()
vect.fit(X_new)
X_new=vect.transform(X_new)
from scipy.sparse import csr_matrix
X_new=csr_matrix(X_new)
csr_matrix.resize(X_new,(1, 116181))
pred=model.predict(X_new)
print(pred)

['non-suicide']
